In [2]:
// SETUP
use std::fs::File;
use std::collections::{HashMap, HashSet};
use std::str::FromStr;
use std::io::{BufRead, BufReader, Error, ErrorKind, Read};
use std::time::Instant;

fn read_lines(path: &str) -> Result<Vec<String>, Error> {
    let file = File::open(path)?; 
    BufReader::new(file).lines().collect()
}

fn read_integers(path: &str) -> Result<Vec<i64>, Error> {
    let mut v = Vec::new();
    for line in read_lines(path)? {
        let n = line   
            .trim() 
            .parse() 
            .map_err(|e| Error::new(ErrorKind::InvalidData, e))?; 
        v.push(n);
    }
    Ok(v)
}

const DATA_ROOT: &str = "../data/";

fn data_file(problem_id: u32) -> String {
    format!("{}/{:02}.txt", DATA_ROOT, problem_id)
}

fn test_file(problem_id: u32, test_id: u32) -> String {
    format!("{}/{:02}t{}.txt", DATA_ROOT, problem_id, test_id)
}

fn extra_file(problem_id: u32, suffix: &str) -> String {
    format!("{}/{:02}.{}.txt", DATA_ROOT, problem_id, suffix)
}

In [3]:
extern crate itertools;
use itertools::Itertools;

fn problem01() {    
    let nums: Vec<i64> = read_integers(&data_file(1)).unwrap();
    let res1: i64 = nums.iter().tuple_windows().map(|(a, b)| (b > a) as i64).sum();
    let res2: i64 = nums.iter().tuple_windows::<(_, _, _)>().map(|(a, b, c)| a + b + c)
        .tuple_windows().map(|(a, b)| (b > a) as i64).sum();
    println!("Answer 1: {}\nAnswer 2: {}\n", res1, res2);
}

problem01();

Answer 1: 1557
Answer 2: 1608



In [4]:
#[derive(Debug, PartialEq, Copy, Clone)]
enum Command {
    Forward(i32),
    Up(i32),
    Down(i32),
}

impl FromStr for Command {
    type Err = ();
    
    fn from_str(s: &str) -> Result<Self, Self::Err> {
        let mut it = s.split(' ');
        let opstr: &str = it.next().ok_or(())?;
        let val: i32 = it.next().ok_or(())?.parse::<i32>().map_err(|_|())?;
        match opstr {
            "forward" => Ok(Command::Forward(val)),
            "up" => Ok(Command::Up(val)),
            "down" => Ok(Command::Down(val)),
            _ => Err(()),
        }
    }
}

fn step1((x, depth): (i32, i32), cmd: &Command) -> (i32, i32) {
    match cmd {
        Command::Up(offs) => (x, depth - offs),
        Command::Down(offs) => (x, depth + offs),
        Command::Forward(offs) => (x + offs, depth),
    }
}

fn step2((x, depth, aim): (i32, i32, i32), cmd: &Command) -> (i32, i32, i32) {
    match cmd {
        Command::Up(offs) => (x, depth, aim - offs),
        Command::Down(offs) => (x, depth, aim + offs),
        Command::Forward(offs) => (x + offs, depth + aim * offs, aim),
    }
}

fn problem02() {
    let commands: Vec<Command> = read_lines(&data_file(2)).unwrap()
            .iter()
            .filter_map(|line| line.parse().ok())
            .collect();
    let pos1 = commands.iter().fold((0i32, 0i32), step1);
    let pos2 = commands.iter().fold((0i32, 0i32, 0i32), step2);
    println!("Answer 1: {}\nAnswer 2: {}\n", pos1.0 * pos1.1, pos2.0 * pos2.1);
}

problem02();

Answer 1: 1728414
Answer 2: 1765720035



In [5]:
fn get_most_common<'a>(nums: impl Iterator<Item = &'a Vec<u8>>, pos: usize) -> u8 {
    let mut n0 = 0;
    let mut n1 = 0;
    for num in nums {
        n0 += (num[pos] == 0) as usize;
        n1 += (num[pos] == 1) as usize;
    }
    if n1 >= n0 {
        1
    } else {
        0
    }
}

fn part1(nums: &Vec<Vec<u8>>) -> i64 {
    let mut lc: i64 = 0;
    let mut mc: i64 = 0;
    for i in 0..nums[0].len() {
        let m = get_most_common(nums.iter(), i) as i64;
        mc = mc * 2 + m;
        lc = lc * 2 + (1 - m);
    }
    mc * lc
}

fn digits_to_decimal(digits: &Vec<u8>)-> i64 {
    digits.iter().fold(0i64, |v, d| v * 2 + *d as i64)
}

fn part2(nums: &Vec<Vec<u8>>) -> i64 {
    let mut mnums: Vec<usize> = (0..nums.len()).collect();
    let mut lnums: Vec<usize> = (0..nums.len()).collect();
    for i in 0..nums[0].len() {
        if mnums.len() > 1 {
            let m = get_most_common(mnums.iter().cloned().map(|x| &nums[x]), i);
            mnums = mnums.iter().cloned().filter(|&x| nums[x][i] == m).collect();
        }
        if lnums.len() > 1 {
            let l = 1 - get_most_common(lnums.iter().cloned().map(|x| &nums[x]), i);
            lnums = lnums.iter().cloned().filter(|&x| nums[x][i] == l).collect();
        }
    }
    digits_to_decimal(&nums[mnums[0]]) * digits_to_decimal(&nums[lnums[0]])
}


fn problem03() {
    let nums: Vec<Vec<u8>> = read_lines(&data_file(3))
        .unwrap()
        .iter()
        .map(|line| line
            .chars()
            .map(|x| x.to_digit(10).unwrap() as u8)
            .collect())
        .collect();
    println!("Answer 1: {}\nAnswer 2: {}", part1(&nums), part2(&nums));
}

problem03();

Answer 1: 2972336
Answer 2: 3368358


In [62]:
type Board = Vec<Vec<i32>>;

fn is_full(board: &Board, pos: usize, is_vert: bool) -> bool {
    for i in 0..board.len() {
        let n = if is_vert {board[i][pos]} else {board[pos][i]};
        if n != -1 {
            return false;
        }
    }
    true
}

fn has_winner(board: &Board) -> bool {
    return (0..board.len())
        .any(|i| is_full(board, i, true) || is_full(board, i, false));
}

fn apply_num(board: &mut Board, num: i32) {
    for mut row in board {
        for i in 0..row.len() {
            if row[i] == num {
                row[i] = -1;
            }
        }
    }
}

fn run_simulation((nums, mut boards): (Vec<i32>, Vec<Board>)) -> Vec<i32> {
    let mut winners = vec![];
    let mut scores = vec![];
    for n in nums {
        for i in 0..boards.len() {
            if winners.contains(&i) {
                continue;
            }
            apply_num(&mut boards[i], n);
            if has_winner(&boards[i]) {
                let s: i32 = boards[i].iter().flatten().filter(|&x| *x != -1).sum();
                winners.push(i);
                scores.push(s * n);
            }
        }
    }
    scores
}

fn parse_game(lines: Vec<String>) -> (Vec<i32>, Vec<Board>) {
    let nums = lines[0].split(",").map(|s| s.parse::<i32>().unwrap()).collect();
    let mut boards = vec![];
    let mut board = vec![];
    for line in &lines[2..] {
        if line.is_empty() {
            boards.push(board);
            board = vec![];
        } else {
            board.push(line.split_whitespace()
                .map(|s| s.parse::<i32>().unwrap())
                .collect());
        }
    }
    boards.push(board);
    (nums, boards)
}

fn problem04() {
    let lines = read_lines(&data_file(4)).unwrap();
    let scores = run_simulation(parse_game(lines));
    println!("Answer 1: {}\nAnswer 2: {}", scores[0], scores.last().unwrap());
}

problem04();

Answer 1: 44736
Answer 2: 1827
